# IMPORTS E CONFIGURAÇÕES

In [1]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zipgit 
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import numpy as np
import pandas as pd  # Manipulação de dados



# Configurações das bibliotecas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

# EXTRAÇÃO DOS DADOS

In [2]:
# Extrair o arquivo excel da pasta ZIP diretamente do site de download
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file, skiprows=6)

# TRANSFORMAÇÃO DOS DADOS

In [3]:
# Renomeando as colunas necessárias
df = df.rename(columns={'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', 'Unnamed: 4': 'LISTAGEM'})[1:-18]
# Preenchendo valores nulos da coluna SEGMENTO com os valores da coluna NOME onde a coluna CÓDIGO FOR vazia
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']
# Removendo linhas com todos as variáveis nulas
df = df.dropna(how='all')
# Preenchendo valores nulos da coluna LISTAGEM com 'Sem classificação'
df.LISTAGEM = df.LISTAGEM.fillna(value='Sem classificação')
# Preenchendo valores nulos com base nas células anteriores
df['SETOR ECONÔMICO'].ffill(inplace=True)
df['SUBSETOR'].ffill(inplace=True)
df['SEGMENTO'].ffill(inplace=True)
# Removendo valores inconsistentes
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]
# Formatando todos os dados
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
# Alterando tipo das variáveis se necessário
df = df.apply(lambda x: x.astype('category') if (x.dtype == 'O') and (x.name not in ['CÓDIGO', 'NOME']) else x)
# Reordenando colunas
df = df[['CÓDIGO', 'NOME', 'SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'LISTAGEM']]
# Reiniciando indexs
df = df.reset_index(drop=True)
# Criando arquivo parquet para acessos posteriores
df.to_parquet(path='./tickers_por_setor.parquet')

In [4]:
display(df)

,CÓDIGO,NOME,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM
0,RRRP,3R PETROLEUM,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
1,CSAN,COSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,ENAT,ENAUTA PART,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
3,RPMG,PET MANGUINH,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",Sem classificação
4,PETR,PETROBRAS,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
...,...,...,...,...,...,...
427,PPAR,POLPAR,Outros,Outros,Outros,Sem classificação
428,PRPT,PROMPT PART,Outros,Outros,Outros,Sem classificação
429,OPSE,SUDESTE S/A,Outros,Outros,Outros,Sem classificação
430,OPTS,SUL 116 PART,Outros,Outros,Outros,Sem classificação


# ANÁLISE EXPLORATÓRIA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CÓDIGO           432 non-null    object  
 1   NOME             432 non-null    object  
 2   SETOR ECONÔMICO  432 non-null    category
 3   SUBSETOR         432 non-null    category
 4   SEGMENTO         432 non-null    category
 5   LISTAGEM         432 non-null    category
dtypes: category(4), object(2)
memory usage: 13.4+ KB


In [6]:
df.memory_usage()

Index               132
CÓDIGO             3456
NOME               3456
SETOR ECONÔMICO     820
SUBSETOR           1856
SEGMENTO           3200
LISTAGEM            804
dtype: int64

# ANÁLISE DESCRITIVA